## Návrh RF filtru pro vstupní obvody přijímače rádiových signálů meteorů

Rádiové signály odražené od stop meteorů jsou frekvenčně modulovány pouze Dopplerovým posuvem proto mají šířku pásma pouze někoklik desítek kHz. Původní vysílaný signál je CW vysílání na frekvenci 143.05 kHz z pohledu RF techniky proto jde o filtr s velmi malou relativní šířkou pásma. 

Jedním z hlavních cílů návrhu je, aby filtr byl poměrně jednoduše vyrobitelný i pro jiné frekvence základního signálu tj. jiný vysílač. Nelze proto zvolit například krystalový nebo dutinový rezonátor, protože kusová výroba takových filtrů pro různé potřebné frekvence by byla velmi nákladná. Konstruční řešení proto musí být realizovatelné ze standardní řady L a C součástek. 

##### Zadání v podobě tolerančního pásma filtru 	
Značení parametrů stejné jako v programu Syntfil https://asinus.feld.cvut.cz/syntfil

Pásmová propust, Chebyshev, 3. řád
f-s = 142.551MHz, f-p = 143.025MHz, $f_p = 143.075MHz$, fs = 143.551MHz, fm = 143.05MHz, Δfp = 50kHz, Δfs = 1MHz, ap = 3dB, as = 60dB

##### Kanonická LC realizace
zakončení: symetrické, R = 50Ω, směr: zepředu, struktura: π

Výsledkem návrhu filtru pomocí běžně používaných návrhových nástrojů je typicky následující struktura, která ale není realizovatelná z reálných součástek. 


![title](Filter_img/GRAVes_syntfil.png)

Takto navržená struktura je kanonickou strukturou, která začíná být prakticky realizovatelná od situace kdy šířka pásma filtru je alespoň polovina pracovní frekvence. Ale i tak jsou potřebné hodnoty součástek těžko zkontruovatelné, protože nutné rozměry vytvářejí potíže s vlastní rezonancí. Existuje ještě několik analyticky správných, ale technicky těžko realizovatelných struktur jako eliptické filtry, Nortonovy transformované filtry atd.

Pro náš případ pásmové propusti lze však využít požadavku na velmi malou šířku pásma, která umožňuje uvažovat o aproximacích platných pro danou frekvenci. Pro změnu impedance mezi jednotlivými bloky filtru tak lze například použít LC články místo trasformátorů z vázaných indukčostí. 

##### Přímo vázaná LC struktura

Filtry tohoto typu se obecně skládají z přizpůsobovacích bloků a rezonátorů. Tato obvodová realizace filtru může využívat obrovské množství různých topologií rezonátorů a přizpůsobovacích obvodů, které lze vzájemně kombinovat. Na následujícím schématu je jedna ze základních struktur.

![Direct coupled Filter](Filter_img/Direct_coupled_RFfilter.png)

Návrh tohoto typu filtru probíhá iterativně v následujících krocích: 

 - Fixní volba hodnoty součástek, jejiž kvalita limituje parametry filtru. (typicky indukčnosti. proto byly zvoleny hodnoty ze [standardní řady](http://www.tme.eu/cz/Document/2e8fe3373db00f79b69db1695f1f0867/cw.pdf) )
 - Ke zvoleným hodnotám indukčností napočítat hodnoty kapacit tak, aby rezonance LC obvodu odpovídala požadovanému pracovnímu kmitočtu.
 - Spočítat hodnoty vazebních členů mezi rezonátory
 - Spočítat hodnoty přizpůsobovacích členů na vstupu a výstupu filtru, aby jejich ipedance odpovídala $\sqrt{Z_{in} Z_{out}}$
 - Schéma odsimulovat a opakovat postup, dokud není dosaženo minimálních ztrát. 
 
Tento postup využívá obecně známé výrazy pro impedanci a rezonanční kmitočet LC obvodu. 

\begin{equation}
Z_k = Q P_K  \omega L = \frac{Q P_k}{\omega C} 
\end{equation}

\begin{equation}
\omega = \frac{1}{\sqrt{LC}} \\ 
\end{equation}

\begin{equation}
Q = \frac{F_c}{BW}
\end{equation}

Kde $Z_k$ je charakteristická impedance rezonátoru. Pro vzájemnou vazbu rezonátorů použijeme kapacity, neboť indukčnosti zbytečně zvyšují cenu filtru a zavádějí riziko vzájemné vazby. 

##### Simulace struktury

Pro návrh hodnot součástek struktuře byl vytvořen model využívající knihovnu [ahkab](https://github.com/ahkab/ahkab)


In [15]:
#  https://github.com/ahkab/ahkab
from ahkab import new_ac, run
from ahkab.circuit import Circuit
from ahkab.plotting import plot_results # calls matplotlib for you
import numpy as np

# Define the circuit
cir = Circuit('Butterworth 143.05 MHz band-pass filter')
cir.add_vsource('V1', 'n1', cir.gnd, dc_value=0., ac_value=1.)
cir.add_resistor('R1', 'n1', 'n2', 50.)
cir.add_inductor('L1', 'n2', 'n3', 100e-09)
cir.add_inductor('L2', 'n3', cir.gnd, 33e-09)
cir.add_capacitor('C2', 'n3', cir.gnd, 43e-12)
cir.add_capacitor('C3', 'n3', 'n4', 3.9e-12)
cir.add_inductor('L3', 'n4', cir.gnd, 33e-09)
cir.add_capacitor('C4', 'n4', cir.gnd, 30e-12)
cir.add_capacitor('C5', 'n4', 'n5', 3.9e-12)
cir.add_inductor('L4', 'n5', cir.gnd, 33e-09)
cir.add_capacitor('C6', 'n5', cir.gnd, 43e-12)
cir.add_inductor('L5', 'n5', 'n6', 100e-09)
cir.add_resistor('R2', 'n6', cir.gnd, 50.)

# Define the analysis
ac1 = new_ac(120e6, 160e6, 1e2, x0=None)

# run it
res = run(cir, ac1)

# plot the results
plot_results('3 Pole Butterworth Direct BPF', [('|Vn6|',"")], res['ac'])

In [16]:
print cir

* Butterworth 143.05 MHz band-pass filter
V1 n1 0 type=vdc value=0.0 vac=1.0 
R1 n1 n2 50
L1 n2 n3 1e-07
L2 n3 0 3.3e-08
C2 n3 0 4.3e-11
C3 n3 n4 3.9e-12
L3 n4 0 3.3e-08
C4 n4 0 3e-11
C5 n4 n5 3.9e-12
L4 n5 0 3.3e-08
C6 n5 0 4.3e-11
L5 n5 n6 1e-07
R2 n6 0 50


In [17]:
from ahkab.plotting import show_plots
show_plots()

Frekvenční charakteristika navrženého pásmového filtru

![Frekvenční odezva navrženého filtru](Filter_img/3pole_butterworth_DBPF.png)

Pro konečnou realizaci bylo zapojení filtru ještě doplněno o DC by-pass, který umožňuje napájení nízkošumovéhe zesilovače umístěného u antény.

![Předpokládaná výsledná realizace filtru](Filter_img/BP01B_schematics.png)


#### Reference 

    http://www.iowahills.com/9RFFiltersPage.html
    http://www.iowahills.com/RF%20Filters/The%20Design%20of%20Direct%20Coupled%20Band%20Pass%20Filters.pdf
